First we are going to parse the annotation file and see what we can learn...

In [1]:
from rdflib import Graph, URIRef
from rdflib.namespace import DCTERMS

# use this URI to identify delayed variables - not the perfect URI, but will do for now
delay_variable_uri = URIRef('https://github.com/nickerso/libcellml-python-utils/properties.rst#delay-variable')
variable_to_delay_uri = URIRef('https://github.com/nickerso/libcellml-python-utils/properties.rst#variable-to-delay')
delay_amount_uri = URIRef('https://github.com/nickerso/libcellml-python-utils/properties.rst#delay-amount')

# a "readout" variable that we maybe want to connect to something external?
timecourse_readout_uri = URIRef('http://identifiers.org/mamo/MAMO_0000031')

model_name = 'delay_test'
annotation_file = f'cellml/{model_name}-updated-ids--annotations.ttl'
g = Graph().parse(annotation_file)

# find all delayed variables
variables_to_delay = []
for d in g.subjects(DCTERMS.type, variable_to_delay_uri):
    # we only expect one delay variable for each variable to delay
    dv = g.value(d, delay_variable_uri)
    d_amount = g.value(d, delay_amount_uri)
    variables_to_delay.append([str(d), str(dv), str(d_amount)])
    
print(variables_to_delay)

# find all timecourse readouts
readout_variables = []
for d in g.subjects(DCTERMS.type, timecourse_readout_uri):
    readout_variables.append(str(d))
    
print(readout_variables)

[['file:///home/farg967/Documents/git_projects/libcellml-python-utils/cellml/delay_test-updated-ids.cellml#b4da66', 'file:///home/farg967/Documents/git_projects/libcellml-python-utils/cellml/delay_test-updated-ids.cellml#b4da64', 'file:///home/farg967/Documents/git_projects/libcellml-python-utils/cellml/delay_test-updated-ids.cellml#b4da69']]
['file:///home/farg967/Documents/git_projects/libcellml-python-utils/cellml/delay_test-updated-ids.cellml#b4da65', 'file:///home/farg967/Documents/git_projects/libcellml-python-utils/cellml/delay_test-updated-ids.cellml#b4da66', 'file:///home/farg967/Documents/git_projects/libcellml-python-utils/cellml/delay_test-updated-ids.cellml#b4da67', 'file:///home/farg967/Documents/git_projects/libcellml-python-utils/cellml/delay_test-updated-ids.cellml#b4da68', 'file:///home/farg967/Documents/git_projects/libcellml-python-utils/cellml/delay_test-updated-ids.cellml#b4da64']


We're going to use the "model" file from the first variable to delay and only continue if all annotations use the same model...

In [2]:
from urllib.parse import urlparse

model_uri = variables_to_delay[0][0]
model_url = urlparse(model_uri)
model_file = model_url.path

delayed_ids = []
for v, dv, d_amount in variables_to_delay:
    url = urlparse(v)
    if url.path != model_file:
        print("found an unexpected model file for variable to delay?!")
        exit
    dv_url = urlparse(dv)
    if dv_url.path != model_file:
        print("found an unexpected model file for delay variable?!")
        exit
    d_amount_url = urlparse(d_amount)
    if d_amount_url.path != model_file:
        print("found an unexpected model file for delay variable?!")
        exit
    delayed_ids.append([url.fragment, dv_url.fragment, d_amount_url.fragment])
    
readout_ids = []
for v in readout_variables:
    url = urlparse(v)
    if url.path != model_file:
        print("found an unexpected model file for readout variable?!")
        exit
    readout_ids.append(url.fragment)

Now we have the model file and the IDs for the variables in that model that we want to do stuff with. So we can parse the model and see if we can find the variables.

In [3]:
import cellml

# on windows getting a leading '/' in the filename which libCellML doesn't like...
fixed_model_file = model_file[0:]

# parse the model in non-strict mode to allow non CellML 2.0 models
model = cellml.parse_model(fixed_model_file, False)

# and make an annotator for this model
from libcellml import Annotator
annotator = Annotator()
annotator.setModel(model)

# map our IDs to the actual variables
annotated_variables = []
for i, dv_i, d_amount_i in delayed_ids:
    # get the variable (will fail if id doesn't correspond to a variable in the model)
    v = annotator.variable(i)
    if v == None:
        print('Unable to find a variable to delay with the id {} in the given model...'.format(i))
        exit
    dv = annotator.variable(dv_i)
    if dv == None:
        print('Unable to find a delay variable with the id {} in the given model...'.format(dv_i))
        exit
    d_amount = annotator.variable(d_amount_i)
    if d_amount == None:
        print('Unable to find a delay variable with the id {} in the given model...'.format(dv_i))
        exit
    annotated_variables.append([[v, dv, d_amount], delay_variable_uri])
    
for i in readout_ids:
    # get the variable (will fail if id doesn't correspond to a variable in the model)
    v = annotator.variable(i)
    if v == None:
        print('Unable to find a readout variable with the id {} in the given model...'.format(i))
        exit
    annotated_variables.append([v, timecourse_readout_uri])

# TODO:
Need to work out how to map the annotations through to the variables in the generated code....

Generate C code for the model.

In [4]:
import os 
model_dir = os.path.dirname(fixed_model_file)

# resolve imports, in non-strict mode
importer = cellml.resolve_imports(model, model_dir, False)
# need a flattened model for analysing
flat_model = cellml.flatten_model(model, importer)

from libcellml import Analyser, AnalyserModel, AnalyserExternalVariable, Generator, GeneratorProfile        

# analyse the model
a = Analyser()

# set the delayed variables as external
external_variable_info = []
for vv, t in annotated_variables:
    if t == delay_variable_uri:
        v = vv[0]
        dv = vv[1]
        flat_variable = flat_model.component(v.parent().name()).variable(v.name())
        flat_delay_variable = flat_model.component(dv.parent().name()).variable(dv.name())
        flat_delay_amount_variable = flat_model.component(dv.parent().name()).variable(d_amount.name())
        aev = AnalyserExternalVariable(flat_variable)
        aev.addDependency(flat_delay_variable)
        aev.addDependency(flat_delay_amount_variable)
        #
        # TODO: really need to work out how to handle other dependencies here to make sure 
        #       all required variables are up to date...
        #
        a.addExternalVariable(aev)
        # keep track of external variable information for use in generating code
        external_variable_info.append({
            'variable': flat_variable,
            'delay_variable': flat_delay_variable,
            'delay_amount_variable': flat_delay_amount_variable,
            'analyser_variable': aev
        })

a.analyseModel(flat_model)
analysed_model = a.model()
print(analysed_model.type())

# get the information for the variables to delay
for ext_variable in external_variable_info:
    ev = ext_variable['variable']
    avs = analysed_model.variables()
    for av in avs:
        v = av.variable()
        if analysed_model.areEquivalentVariables(v, ext_variable['variable']):
            ext_variable['index'] = av.index()
        if v.name() == ext_variable['delay_amount_variable'].name():
            ext_variable['delay_index'] = av.index()


no unresolved imports.
2


In [5]:

# generate code from the analysed model
g = Generator()
# using the C profile to generate C code
profile = GeneratorProfile(GeneratorProfile.Profile.C)
profile.setInterfaceFileNameString(f'{model_name}.h')
g.setProfile(profile)
g.setModel(analysed_model)

circularBufferHeader = f"""
#include <stdlib.h>
#include <mutex>
#include <memory>
#include <map>
template <typename T>
class circular_buffer {{
public:
	explicit circular_buffer(size_t size) :
		buf_(std::unique_ptr<T[]>(new T[size])),
		max_size_(size) {{}};

	void put(T item);
	T get();
	void reset();
	bool empty() const;
	bool full() const;
	size_t capacity() const;
	size_t size() const;

private:
	std::mutex mutex_;
	std::unique_ptr<T[]> buf_;
	size_t head_ = 0;
	size_t tail_ = 0;
	const size_t max_size_;
	bool full_ = 0;
}};
"""

circularBuffer = f"""
class circular_buffer(size_t size)
{{
	//empty constructor
}}

void reset()
{{
	std::lock_guard<std::mutex> lock(mutex_);
	head_ = tail_;
	full_ = false;
}}

bool empty() const
{{
	//if head and tail are equal, we are empty
	return (!full_ && (head_ == tail_));
}}

bool full() const
{{
	//If tail is ahead the head by 1, we are full
	return full_;
}}

size_t capacity() const
{{
	return max_size_;
}}

size_t size() const
{{
	size_t size = max_size_;

	if(!full_)
	{{
		if(head_ >= tail_)
		{{
			size = head_ - tail_;
		}}
		else
		{{
			size = max_size_ + head_ - tail_;
		}}
	}}

	return size;
}}

void put(T item)
{{
	std::lock_guard<std::mutex> lock(mutex_);

	buf_[head_] = item;

	if(full_)
	{{
		tail_ = (tail_ + 1) % max_size_;
  }}

	head_ = (head_ + 1) % max_size_;

	full_ = head_ == tail_;
}}

T get()
{{
	std::lock_guard<std::mutex> lock(mutex_);

    if(empty())
	{{
		return T();
    }}

	//Read data and advance the tail (we now have a free space)
	auto val = buf_[tail_];
	full_ = false;
	tail_ = (tail_ + 1) % max_size_;

	return val;
}}

"""

storeEVSingletonHeader = f"""
class EVSingleton  {{
  private:
    static EVSingleton obj;
  public:
    std::map<std::string, boost::any> circular_buffer_dict;
    // circular_buffer<uint32_t> circular_buffer_dict ;
    static getInstance(double dt);
    circular_buffer<uint32_t> var_circular_buffer;\n
}};
"""

# generate a global singleton class to store external variables
storeEVSingleton = f"""

// creating a singleton design pattern by using a private constructor
class EVSingleton
  {{
  private EVSingleton(double dt) 
  {{
  // Here I need to initialise circular buffers for each external variable   
"""

for ext_variable in external_variable_info:
    print(ext_variable.keys())
    index = ext_variable['index']
    v = ext_variable['variable']
    delay_index = ext_variable['delay_index']
    storeEVSingleton += f'  double buffer_size = static_cast<double>(variables[{delay_index}])/dt;\n' 
    storeEVSingleton += f'  circular_buffer<uint32_t> var_circular_buffer(buffer_size);\n'
    storeEVSingleton += f'  circular_buffer_dict[{index}] = var_circular_buffer;\n'
    storeEVSingleton += f'  }}\n'

storeEVSingleton += f""" 

  EVSingleton::getInstance(double dt)
  {{
  if (obj==null)
    obj = new Singleton(dt);
  return obj;
  }}

  void EVSingleton::storeVariable(int index, double value)
  {{
    // Here I need to store the value in the correct circular buffer    
    circular_buffer_dict[index].put(value);

  }}

  double EVSingleton::getVariable(int index)
  // Here I need to get the value from the correct circular buffer
  return circular_buffer_dict[index].get();

}}
"""

# and generate a function to compute external variables
computeEV = f"""
double externalVariable(double voi, double *states, double *variables, size_t index)
{{
  
"""
for ext_variable in external_variable_info:
    print(ext_variable.keys())
    index = ext_variable['index']
    v = ext_variable['variable']
    delay_index = ext_variable['delay_index']
    computeEV += f'  if (index == {index}) {{\n'
    computeEV += f'    if (voi < variables[{delay_index}]) {{\n'
    computeEV += f'      evi.storeVariable({index}, variables[{index}]);\n'
    computeEV += f'      return 0.0;\n'
    computeEV += f'    }} else {{;\n'
    computeEV += f'    // save the current value of the variable to the circle buffer\n'
    computeEV += f'      evi.storeVariable({index}, variables[{index}]);\n'
    computeEV += f'      evi.getVariable({index}, variables[{index}]);\n'
    computeEV += f'      return variables[{index}];\n'
    computeEV += f'    }};\n'
    computeEV += f'  }}\n'

    # TODO also create an if statement for coupling variables with external models
    # if the variable is of coupling type, not a delay variable.

computeEV += f"""
  return 0.0;
}}
"""


dict_keys(['variable', 'delay_variable', 'delay_amount_variable', 'analyser_variable', 'delay_index', 'index'])
dict_keys(['variable', 'delay_variable', 'delay_amount_variable', 'analyser_variable', 'delay_index', 'index'])


In [6]:

mainScript = """
#include <stdio.h>
int main(void){
    double voi = 0.0;
    double end_time = 10.0;
    double dt = 0.01;
    double * states = createStatesArray();
    double * rates = createStatesArray(); // same size as states, should really be different function
    double * variables = createVariablesArray();



    global EVSingleton evi.getInstance(dt);


    initialiseVariables(voi, states, variables, externalVariable);
    computeComputedConstants(variables);


    while (voi < end_time) {
        computeVariables(voi, states, rates, variables, externalVariable);
        computeRates(voi, states, rates, variables, externalVariable);

        // simple forward Euler integration
        for (size_t i = 0; i < STATE_COUNT; ++i) {
            states[i] = states[i] + dt * rates[i];
        }
        
        voi += dt;   
    }
    
    
    computeVariables(voi, states, rates, variables, externalVariable);
    computeRates(voi, states, rates, variables, externalVariable);

    printf("Final values:");
    printf("  time: ");
    printf("%f", voi);
    printf("  states:");
    for (size_t i = 0; i < STATE_COUNT; ++i) {
        printf("%f\\n", states[i]);
    }
    printf("  variables:");
    for (size_t i = 0; i < VARIABLE_COUNT; ++i) {
        printf("%f\\n", variables[i]);
    }

return 0;
}
"""


print('compute external variables implementation:')
print(computeEV)

with open(f'C_models/{model_name}.h', 'w') as f:

    f.write(g.interfaceCode())
    f.write(circularBufferHeader)
    f.write(storeEVSingletonHeader)


# and save to a file
with open(f'C_models/{model_name}.c', 'w') as f:

    f.write(g.implementationCode())
    f.write(circularBuffer)
    f.write(storeEVSingleton)
    f.write(computeEV)
    f.write(mainScript)


compute external variables implementation:

double externalVariable(double voi, double *states, double *variables, size_t index)
{
  
  if (index == 3) {
    if (voi < variables[0]) {
      evi.storeVariable(3, variables[3]);
      return 0.0;
    } else {;
    // save the current value of the variable to the circle buffer
      evi.storeVariable(3, variables[3]);
      evi.getVariable(3, variables[3]);
      return variables[3];
    };
  }

  return 0.0;
}

